In [ ]:
import pandas
import numpy as np
import pandas as pd

In [ ]:
data_path = '/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction_20211110'
admission_data_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_unit_dataset/data/stroke_registry/post_hoc_modified/stroke_registry_post_hoc_modified.xlsx'
patient_selection_path = '/Users/jk1/temp/opsum_extration_output/high_frequency_data_patient_selection.csv'

In [ ]:
from preprocessing.variable_assembly.variable_database_assembly import assemble_variable_database

feature_df = assemble_variable_database(data_path, admission_data_path, patient_selection_path)

In [ ]:
from preprocessing.variable_assembly.relative_timestamps import transform_to_relative_timestamps

restricted_feature_df = transform_to_relative_timestamps(feature_df, drop_old_columns=False, restrict_to_time_range=True)

In [ ]:
from preprocessing.normalisation.normalisation import normalise_data

normalised_restricted_feature_df = normalise_data(restricted_feature_df, verbose=True)

In [ ]:
from preprocessing.encoding_categorical_variables.encode_categorical_variables import encode_categorical_variables

cat_encoded_normalised_restricted_feature_df = encode_categorical_variables(normalised_restricted_feature_df)

In [ ]:
cat_encoded_normalised_restricted_feature_df.head()

In [ ]:
cat_encoded_normalised_restricted_feature_df['relative_sample_date_hourly_cat'] = np.floor(cat_encoded_normalised_restricted_feature_df['relative_sample_date'])

In [ ]:
cat_encoded_normalised_restricted_feature_df.head()

In [ ]:
for variable in cat_encoded_normalised_restricted_feature_df.sample_label.unique():
    print(f"'{variable}',")

In [ ]:
variables_to_down_sample = [
    'NIHSS',
    'oxygen_saturation',
    'systolic_blood_pressure',
    'diastolic_blood_pressure',
    'mean_blood_pressure',
    'heart_rate',
    'respiratory_rate'
]

In [ ]:
# find hourly median value for NIHSS
median_NIHSS = cat_encoded_normalised_restricted_feature_df[cat_encoded_normalised_restricted_feature_df.sample_label == 'NIHSS'].groupby([
    'case_admission_id',
    'relative_sample_date_hourly_cat'
])['value'].median().reset_index()
median_NIHSS

In [ ]:
cat_encoded_normalised_restricted_feature_df[cat_encoded_normalised_restricted_feature_df.sample_label == 'NIHSS'].groupby([
    'case_admission_id',
    'relative_sample_date_hourly_cat'
])['value'].min().reset_index()

In [ ]:
cat_encoded_normalised_restricted_feature_df_essential_columns = cat_encoded_normalised_restricted_feature_df[[
    'case_admission_id',
    'relative_sample_date_hourly_cat',
    'sample_label',
    'source',
    'value'
]]

In [ ]:
for variable in variables_to_down_sample:
    print(f"{variable}")
    median_variable_df = cat_encoded_normalised_restricted_feature_df[cat_encoded_normalised_restricted_feature_df.sample_label == variable].groupby([
        'case_admission_id',
        'relative_sample_date_hourly_cat'
    ])['value'].median().reset_index()
    median_variable_df['sample_label'] = f'median_{variable}'
    max_variable_df = cat_encoded_normalised_restricted_feature_df[cat_encoded_normalised_restricted_feature_df.sample_label == variable].groupby([
        'case_admission_id',
        'relative_sample_date_hourly_cat'
    ])['value'].max().reset_index()
    max_variable_df['sample_label'] = f'max_{variable}'
    min_variable_df = cat_encoded_normalised_restricted_feature_df[cat_encoded_normalised_restricted_feature_df.sample_label == variable].groupby([
        'case_admission_id',
        'relative_sample_date_hourly_cat'
    ])['value'].min().reset_index()
    min_variable_df['sample_label'] = f'min_{variable}'
    temp_df = pd.concat([median_variable_df, max_variable_df, min_variable_df], axis=0)
    # all variables to downsample are from EHR
    temp_df['source'] = 'EHR'
    cat_encoded_normalised_restricted_feature_df_essential_columns = cat_encoded_normalised_restricted_feature_df_essential_columns.append(temp_df)
    # drop all rows of sample label variable
    cat_encoded_normalised_restricted_feature_df_essential_columns = cat_encoded_normalised_restricted_feature_df_essential_columns[cat_encoded_normalised_restricted_feature_df_essential_columns.sample_label != variable]


In [ ]:
cat_encoded_normalised_restricted_feature_df_essential_columns.sample_label.unique()

In [ ]:
all_other_vars = [sample_label for sample_label in cat_encoded_normalised_restricted_feature_df.sample_label.unique()
                                if sample_label not in variables_to_down_sample]

In [ ]:
# for all other variables, when more than one sample per hour is present, take the median
for variable in all_other_vars:
    print(f"{variable}")
    median_variable_df = cat_encoded_normalised_restricted_feature_df[cat_encoded_normalised_restricted_feature_df.sample_label == variable].groupby([
        'case_admission_id',
        'relative_sample_date_hourly_cat'
    ])['value'].median().reset_index()
    median_variable_df['sample_label'] = f'{variable}'
    median_variable_df['source'] = cat_encoded_normalised_restricted_feature_df[cat_encoded_normalised_restricted_feature_df.sample_label == variable]['source'].mode()[0]
    # drop old rows of the variable
    cat_encoded_normalised_restricted_feature_df_essential_columns = cat_encoded_normalised_restricted_feature_df_essential_columns[cat_encoded_normalised_restricted_feature_df_essential_columns.sample_label != variable]
    cat_encoded_normalised_restricted_feature_df_essential_columns = cat_encoded_normalised_restricted_feature_df_essential_columns.append(median_variable_df)


In [ ]:
# count number of values per case_admission_id, relative_sample_date_hourly_cat, sample_label
cat_encoded_normalised_restricted_feature_df_essential_columns.groupby(['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label']).count().reset_index()

In [ ]:
cat_encoded_normalised_restricted_feature_df_essential_columns.groupby(['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label']).count().reset_index().value.unique()

In [ ]:
from preprocessing.resample_to_time_bins.resample_to_hourly_features import resample_to_hourly_features

resampled_df = resample_to_hourly_features(cat_encoded_normalised_restricted_feature_df)
resampled_df.head()

In [ ]:
resampled_df.groupby(['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label']).count().reset_index()

In [ ]:
(cat_encoded_normalised_restricted_feature_df_essential_columns == resampled_df).all().all()